In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd
from Offline_test import *

In [2]:
"""model parameters"""
D = 1e2
n_site = 12
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.3e6
path = 'result/transferRL_' + str(round(n_site)) + 'patch'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)


model have been loaded


In [5]:
tic = time.time()
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 22585916.0, E: 84.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-171.96
 Time: 0 | S: 22585916.0, E: 84.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-171.96
 Time: 0 | new_e:39.7, cum_e:39.7, new_i:35.5, cum_i:35.5
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 22585849.9, E: 110.9, A: 0.0, I: 35.5, D: 0.0, R: 3.8, Budget: 2300000.00, reward:-415.76
 Time: 1 | S: 22585851.0, E: 108.0, A: 0.0, I: 35.5, D: 0.0, R: 3.0, Budget: 2300000.00, reward:-415.76
 Time: 1 | new_e:55.7, cum_e:95.4, new_i:49.0, cum_i:84.5
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 2 | S: 22585757.0, E: 149.7, A: 0.0, I: 84.4, D: 0.0, R: 8.8, Budget: 230

[0.15680885377598894, 0.15744066759328484]
[0.8034551199076214, 0.8040165065792887]
[0.10548717148734943, 0.10549383960518532]
[0.043037250723514, 0.043040540514307175]
 Time: 11 | S: 22410802.9, E: 120.2, A: 858.0, I: 1434.9, D: 3.8, R: 172780.2, Budget: 2160495.38, reward:-8241.41
 Time: 11 | S: 22410837.0, E: 125.0, A: 858.0, I: 1434.9, D: 3.8, R: 172767.0, Budget: 2160495.38, reward:-8241.41
 Time: 11 | new_e:56.3, cum_e:1837.9, new_i:142.3, cum_i:1953.1
[[9.9713552e-01 1.7413618e-05 1.5740473e-04 9.8126400e-01 1.7136145e-04
  1.5335464e-06 9.9992430e-01 9.9992704e-01 9.9999869e-01 1.4248207e-07
  1.4164347e-02 9.2661518e-05]
 [9.9999964e-01 9.9378502e-01 2.6100311e-06 9.9999845e-01 9.9995553e-01
  3.6042997e-01 7.2897360e-02 9.9998295e-01 1.4087936e-01 1.1601122e-05
  9.0941846e-01 5.2104032e-01]
 [9.9999785e-01 9.9999821e-01 9.9999893e-01 9.9999976e-01 9.9999905e-01
  9.9999940e-01 9.9999893e-01 9.9999905e-01 9.9999893e-01 9.9999881e-01
  9.9999833e-01 9.9999976e-01]
 [9.9999893e

[0.1569016899490211, 0.15692385896022337]
[0.803466734692762, 0.8034668255145769]
[0.10549320875961692, 0.10549320911441201]
[0.04304045310143719, 0.0430404533526961]
 Time: 18 | S: 22152827.0, E: 22.6, A: 411.3, I: 1253.9, D: 7.9, R: 431477.4, Budget: 1946960.87, reward:-9261.53
 Time: 18 | S: 22152827.0, E: 24.0, A: 411.3, I: 1253.9, D: 7.9, R: 431454.0, Budget: 1946960.87, reward:-9261.53
 Time: 18 | new_e:10.5, cum_e:2021.0, new_i:53.5, cum_i:2559.9
[[6.3507420e-01 1.0098656e-07 5.0980316e-07 9.9999988e-01 9.9706346e-01
  5.5056764e-05 8.3618736e-01 9.9999917e-01 9.9991226e-01 4.4824119e-06
  2.4601840e-07 7.1828673e-04]
 [1.0000000e+00 9.5791686e-01 1.3313368e-04 9.9976605e-01 9.9999928e-01
  9.9999917e-01 9.9999774e-01 9.9997044e-01 9.9998260e-01 1.3807781e-04
  9.7464585e-01 8.6473995e-01]
 [1.0000000e+00 1.0000000e+00 9.9998879e-01 1.0000000e+00 9.9999988e-01
  1.0000000e+00 9.9999523e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00
  9.9999368e-01 1.0000000e+00]
 [9.9999940e-01 1

[0.15690311933439233, 0.15690312186581473]
[0.8034667398481936, 0.8034667398659276]
[0.10549320907626579, 0.10549320907636622]
[0.0430404533460695, 0.04304045334621723]
 Time: 25 | S: 21934340.1, E: 3.5, A: 153.5, I: 886.5, D: 11.5, R: 650605.0, Budget: 1732526.27, reward:-9606.05
 Time: 25 | S: 21934316.0, E: 3.0, A: 153.5, I: 886.5, D: 11.5, R: 650580.0, Budget: 1732526.27, reward:-9606.05
 Time: 25 | new_e:1.6, cum_e:2051.1, new_i:17.9, cum_i:2772.0
[[9.2440259e-01 4.5783068e-05 2.6630405e-09 9.9999964e-01 9.9999058e-01
  7.0263555e-09 5.5787183e-04 9.9999034e-01 9.9999321e-01 1.6627045e-08
  3.4848423e-11 4.2713928e-06]
 [1.0000000e+00 9.9127138e-01 1.3285502e-07 9.9975246e-01 9.9996018e-01
  9.9995482e-01 9.9999595e-01 9.9982446e-01 9.9994195e-01 4.6886328e-08
  9.9986124e-01 9.9999881e-01]
 [1.0000000e+00 1.0000000e+00 9.9999738e-01 1.0000000e+00 9.9999976e-01
  1.0000000e+00 9.9999189e-01 9.9997127e-01 1.0000000e+00 1.0000000e+00
  9.9999547e-01 1.0000000e+00]
 [9.9999714e-01 9.

[0.15690312184021066, 0.15690312184022423]
[0.8034667398620627, 0.8034667398625148]
[0.10549320907636123, 0.10549320907636124]
[0.04304045334607871, 0.04304045334607872]
 Time: 32 | S: 21698888.2, E: 0.7, A: 51.7, I: 626.8, D: 14.7, R: 886417.9, Budget: 1507873.64, reward:-9819.69
 Time: 32 | S: 21698865.0, E: 0.0, A: 51.7, I: 626.8, D: 14.7, R: 886389.0, Budget: 1507873.64, reward:-9819.69
 Time: 32 | new_e:0.4, cum_e:2055.6, new_i:5.9, cum_i:2841.3
[[9.9753666e-01 6.9923866e-01 1.3683769e-05 9.9999380e-01 9.9998617e-01
  2.6174291e-06 2.2093642e-04 9.9999881e-01 9.9805403e-01 1.9022757e-08
  8.4002323e-08 3.1849657e-05]
 [9.9997377e-01 9.9949038e-01 7.0961250e-05 9.9998844e-01 9.9999821e-01
  9.9999833e-01 9.9999774e-01 9.9915254e-01 9.9998581e-01 7.0105854e-09
  9.9999642e-01 1.0000000e+00]
 [9.9999988e-01 9.9999940e-01 9.9999702e-01 9.9999988e-01 9.9999988e-01
  9.9999976e-01 9.9999642e-01 9.9999857e-01 1.0000000e+00 9.9999928e-01
  9.9999750e-01 1.0000000e+00]
 [9.9999571e-01 9.99

...

[0.1569031218402239, 0.15690312184022395]
[0.8034667398623345, 0.8034667398623391]
[0.10549320907636123, 0.10549320907636123]
[0.04304045334607871, 0.04304045334607871]
 Time: 60 | S: 21032615.4, E: 0.0, A: 0.7, I: 205.6, D: 23.8, R: 1553154.6, Budget: 661266.68, reward:-10355.39
 Time: 60 | S: 21032593.0, E: 0.0, A: 0.7, I: 205.6, D: 23.8, R: 1553138.0, Budget: 661266.68, reward:-10355.39
 Time: 60 | new_e:0.0, cum_e:2059.9, new_i:0.1, cum_i:2878.1
[[1.3585071e-04 9.9986601e-01 9.9999607e-01 3.1314234e-05 1.0000000e+00
  2.6766077e-08 7.7422113e-09 2.3861080e-06 3.1404186e-05 1.2993540e-05
  3.5407166e-09 9.9999571e-01]
 [1.0000000e+00 9.9998343e-01 9.9885821e-01 9.9999976e-01 9.9975675e-01
  9.9999976e-01 9.9996042e-01 9.9999940e-01 6.8926478e-05 8.0020398e-01
  1.0000000e+00 9.9999976e-01]
 [1.0000000e+00 1.0000000e+00 9.9999917e-01 9.9994242e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00 1.0000000e+00
  9.9999976e-01 9.9999762e-01]
 [9.9999928e-01 1.000

[0.1569031218402239, 0.15690312184022395]
[0.8034667398623362, 0.8034667398623381]
[0.10549320907636123, 0.10549320907636123]
[0.04304045334607871, 0.04304045334607871]
 Time: 67 | S: 20927165.0, E: 0.0, A: 0.2, I: 110.4, D: 24.2, R: 1658700.1, Budget: 466415.00, reward:-10380.05
 Time: 67 | S: 20927139.0, E: 0.0, A: 0.2, I: 110.4, D: 24.2, R: 1658686.0, Budget: 466415.00, reward:-10380.05
 Time: 67 | new_e:0.0, cum_e:2059.9, new_i:0.0, cum_i:2878.4
[[1.5612178e-06 9.9999559e-01 9.9999571e-01 4.8292377e-06 1.0000000e+00
  9.9860698e-01 7.1821079e-05 1.6244456e-06 6.9988901e-06 1.4653424e-03
  3.3144462e-03 9.9999928e-01]
 [1.0000000e+00 9.9999988e-01 9.9999928e-01 1.0000000e+00 9.9999988e-01
  1.0000000e+00 9.9999988e-01 1.0000000e+00 2.5451968e-07 9.9933052e-01
  1.0000000e+00 2.0046634e-04]
 [1.0000000e+00 1.0000000e+00 9.9999988e-01 9.9998796e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9999893e-01]
 [1.0000000e+00 1.000

[0.1569031218402239, 0.15690312184022395]
[0.803466739862337, 0.8034667398623377]
[0.10549320907636123, 0.10549320907636123]
[0.04304045334607871, 0.04304045334607871]
 Time: 74 | S: 20800761.1, E: 0.0, A: 0.1, I: 51.3, D: 24.4, R: 1785163.1, Budget: 272686.94, reward:-10391.43
 Time: 74 | S: 20800733.0, E: 0.0, A: 0.1, I: 51.3, D: 24.4, R: 1785152.0, Budget: 272686.94, reward:-10391.43
 Time: 74 | new_e:0.0, cum_e:2059.9, new_i:0.0, cum_i:2878.5
[[2.9030960e-07 9.9997973e-01 1.7370870e-04 1.6555706e-06 9.9999988e-01
  9.9999511e-01 9.9862349e-01 1.1070629e-09 8.8421528e-09 4.6283819e-02
  1.8224161e-03 9.9999726e-01]
 [1.0000000e+00 1.0000000e+00 9.9738210e-01 1.0000000e+00 1.0000000e+00
  9.9999952e-01 9.9999595e-01 1.0000000e+00 2.7064018e-09 3.9799726e-03
  1.0000000e+00 5.7925353e-09]
 [1.0000000e+00 1.0000000e+00 9.9999976e-01 9.9999964e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01
  1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.000000

[0.1569031218402239, 0.15690312184022395]
[0.8034667398623371, 0.8034667398623377]
[0.10549320907636123, 0.10549320907636123]
[0.04304045334607871, 0.04304045334607871]
 Time: 81 | S: 20750443.8, E: 0.0, A: 0.0, I: 43.6, D: 24.7, R: 1835487.9, Budget: 80414.42, reward:-10409.85
 Time: 81 | S: 20750418.0, E: 0.0, A: 0.0, I: 43.6, D: 24.7, R: 1835473.0, Budget: 80414.42, reward:-10409.85
 Time: 81 | new_e:0.0, cum_e:2059.9, new_i:0.0, cum_i:2878.5
[[1.8586248e-07 6.1685331e-03 3.7974772e-08 2.0849548e-04 9.9981111e-01
  1.1103431e-04 1.5423556e-03 6.0677119e-02 1.1516446e-01 1.6018316e-01
  6.9162452e-05 9.9995196e-01]
 [1.0000000e+00 1.0000000e+00 9.9999940e-01 9.9999988e-01 1.0000000e+00
  1.0000000e+00 9.9999988e-01 1.0000000e+00 9.5732510e-03 4.6184261e-05
  1.0000000e+00 1.3416712e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000

 Time: 88 | S: 20790560.3, E: 25.2, A: 0.0, I: 64.7, D: 25.0, R: 1795324.7, Budget: 8555.46, reward:-10632.92
 Time: 88 | S: 20790548.0, E: 6.0, A: 0.0, I: 64.7, D: 25.0, R: 1795311.0, Budget: 8555.46, reward:-10632.92
 Time: 88 | new_e:16.9, cum_e:2106.3, new_i:13.9, cum_i:2915.2
[[4.23702522e-06 2.66507438e-08 5.01462977e-12 9.99185145e-01
  1.74379984e-05 7.30052307e-07 1.18753585e-08 1.42395899e-01
  1.14067448e-02 4.57918458e-02 1.40431189e-06 2.91797142e-11]
 [1.00000000e+00 4.89745848e-03 9.99999762e-01 7.97286071e-03
  9.99981403e-01 8.21551919e-01 9.97741461e-01 1.00000000e+00
  9.97267127e-01 1.53285029e-07 1.00000000e+00 2.74044782e-01]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 9.99998093e-01
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 9.99989629e-01 9.99999881e-01 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  9.99999523e-01 9.99908566e-01 6.15459859e-01 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1

In [6]:
(time.time() - tic) / ( T - start_time + 1)

57.89438312754911